In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
!pip install -q torch==1.4

ERROR: Could not find a version that satisfies the requirement torch==1.4 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1)
ERROR: No matching distribution found for torch==1.4


In [5]:
import requests
from PIL import Image
import torch
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from IPython.display import display_markdown

In [3]:
print("CUDA available:", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")

CUDA available: True
Device name: NVIDIA GeForce RTX 4080 SUPER


In [10]:
# Use a pipeline as a high-level helper
from transformers import pipeline

messages = [
    {"role": "user", "content": "Who are you?"},
]
model = pipeline("text-generation", model="google/gemma-2-2b-it")
model(messages)

model-00001-of-00002.safetensors:  32%|###1      | 1.59G/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Device set to use cuda:0
The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': "I am Gemma, an open-weights AI assistant. I'm a large language model, trained"}]}]

In [6]:
# Reasoning Module
def reasoning(input_query):
    # Prompt template for reasoning
    prompt = f"""
You are a reasoning assistant. Break down the problem logically.
Question: {input_query}

Think step-by-step:
1.
"""
    response = model(prompt, max_length=100, num_return_sequences=1)
    reasoning_output = response[0]['generated_text']
    return reasoning_output

# Acting Module
def acting(input_query):
    if "circle" in input_query.lower():
        # Parse radius from the input query
        radius = float(input_query.split("radius")[1].split()[0])
        area = 3.14159 * radius**2
        return f"The calculated area is {area:.2f} square units."
    elif "rectangle" in input_query.lower():
        # Static values for example purposes
        width = 5
        height = 10
        area = width * height
        return f"The calculated area is {area} square units."
    return "Action not defined for this query."

# ReAct Integration
def react(input_query):
    # Step 1: Reasoning
    reason = reasoning(input_query)
    # Step 2: Acting
    action = acting(input_query)
    # Combine results
    return f"Reasoning:\n{reason}\n\nAction:\n{action}"

# Example Usage
if __name__ == "__main__":
    query = "What is the area of a circle with radius 7 cm?"
    result = react(query)
    print(result)


RuntimeError: Failed to import transformers.models.gpt2.modeling_tf_gpt2 because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.